In [1]:
import pandas as pd

In [2]:
qa_df = pd.read_parquet("../korean_embedding/data/qa.parquet")
qa_df.head()

,retrieval_gt,qid,query,generation_gt
0,[[c68a1103-7cb5-4f86-a46a-5f75fb88cfc2]],4ce77353-5441-4d2d-94e4-e837fcb19e25,일본 정부가 AI 위험을 줄이기 위해 어떤 조치를 취했나요?,"[일본 정부는 AI 위험을 줄이기 위해 기술적 해결책을 중요시하며, AI 글로벌 파..."
1,[[d55e6e70-8741-4063-9c1d-cc7357b9c268]],4ec41559-67d1-45aa-858e-9747b6fb6bbd,마이크로소프트가 프랑스에 투자한 금액은 얼마인가요?,[마이크로소프트가 프랑스에 투자한 금액은 40억 유로입니다.]
2,[[61097ef9-739d-488a-9ce4-802433a349ba]],fc74a5be-098b-4563-9bbe-bd1fc23f45df,ICML 2025 행사에서 다루는 주요 영역은 무엇인가요?,"[ICML 2025 행사에서는 머신러닝, 딥러닝, 최적화, 신뢰할 수 있는 머신러닝..."
3,[[581b4311-2236-44dd-80b0-e02c6906984b]],14408056-937f-44a1-878e-e73c3b5b5f09,구글이 AI 기회 기금의 1차 수혜자로 선정한 단체는 무엇인가요?,[구글이 AI 기회 기금의 1차 수혜자로 선정한 단체는 재향군인 ·군인가족협회와 굿...
4,[[db2b35e9-4f60-400b-bcf2-6443e978f621]],68713781-37e8-44ac-ab49-8e57b9402aec,GPT-4o는 어떤 성능을 보여주었나요?,"[GPT-4o는 GPT-4 터보와 동일한 성능을 보여주었으며, 다국어, 오디오, 이..."


In [3]:
qa_df = qa_df[["query", "generation_gt"]]
qa_df.columns = ["question", "ground_truth"]
qa_df["ground_truth"] = qa_df["ground_truth"].apply(lambda x: x[0])
qa_df.head()

,question,ground_truth
0,일본 정부가 AI 위험을 줄이기 위해 어떤 조치를 취했나요?,"일본 정부는 AI 위험을 줄이기 위해 기술적 해결책을 중요시하며, AI 글로벌 파트..."
1,마이크로소프트가 프랑스에 투자한 금액은 얼마인가요?,마이크로소프트가 프랑스에 투자한 금액은 40억 유로입니다.
2,ICML 2025 행사에서 다루는 주요 영역은 무엇인가요?,"ICML 2025 행사에서는 머신러닝, 딥러닝, 최적화, 신뢰할 수 있는 머신러닝 ..."
3,구글이 AI 기회 기금의 1차 수혜자로 선정한 단체는 무엇인가요?,구글이 AI 기회 기금의 1차 수혜자로 선정한 단체는 재향군인 ·군인가족협회와 굿윌...
4,GPT-4o는 어떤 성능을 보여주었나요?,"GPT-4o는 GPT-4 터보와 동일한 성능을 보여주었으며, 다국어, 오디오, 이미..."


In [ ]:
from langchain_community.chat_models import ChatOllama

gemma2 = ChatOllama(model="gemma2")

In [ ]:
qa_df["answer"] = qa_df["question"].apply(lambda x: rag_chain.invoke(x))
qa_df["contexts"] = qa_df["question"].apply(
    lambda x: [d.page_content for d in retriever.get_relevant_documents(x)]
)
qa_df.head()

In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(qa_df)
dataset[0]

In [ ]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEmbeddings

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=3,
)
langchain_llm = LangchainLLMWrapper(llm)

model_name = "BAAI/bge-m3"
model_kwargs = {"device": "mps"}
encode_kwargs = {"normalize_embeddings": False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

gemma2_result = evaluate(
    dataset,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_recall,
        context_precision,
    ],
    llm=langchain_llm,
    embeddings=embeddings,
    raise_exceptions=False,
)

gemma2_result

In [ ]:
gemma2_df = pd.DataFrame(list(gemma2_result.items()), columns=["Metric", "Gemma2"])
llama3_df = pd.DataFrame(list(gemma2_result.items()), columns=["Metric", "Llama3"])

df_result = pd.merge(gemma2_df, llama3_df, on="Metric")
df_result